In [1]:
import pandas as pd
import numpy as np
from keras.layers import Activation, Dense, Dropout, SpatialDropout1D,Input,Masking,Bidirectional, TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM,GRU
from keras.models import Sequential, Model
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from random import seed
#seed(1)
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model

#defining the early stopping criteria
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,restore_best_weights=True,patience=3)


Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
gs = pd.read_csv('data_8th_june.csv')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15,16,17,25,32,33,35,38,39,41,42,43,44,45,46,47,48,53,55,57,68,69) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
gs.drop('Unnamed: 0',axis=1,inplace=True)
gs.drop('Unnamed: 0.1',axis=1,inplace=True)

In [4]:
cols_to_use = ['uhid','pulserate', 'dischargestatus']

In [5]:
gd = gs[cols_to_use]

In [6]:
def range_finder(x):
    length = x
    fractional = (x/15.0) - math.floor(x/15.0)
    return int(round(fractional*15))

In [7]:
import math
final_df = pd.DataFrame(columns=gd.columns)
for i in gd.uhid.unique():
    x = gd[gd['uhid']==i]
    x = x[range_finder(len(x)):len(x)]
    
    final_df = final_df.append(x,ignore_index=True)
    

In [8]:
final_df.fillna(-999,inplace=True)

In [9]:
train = final_df[:515340]
test = final_df[515340:]

In [10]:
y_train = train['dischargestatus']
X_train = train.drop('dischargestatus',axis=1)
X_train = X_train.drop('uhid',axis=1)
#X_train = X_train.drop('visittime',axis=1)

y_test = test['dischargestatus']
X_test = test.drop('dischargestatus',axis=1)
X_test = X_test.drop('uhid',axis=1)
#X_test = X_test.drop('startdate',axis=1)

In [11]:
#converting the data into a numpy array
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

ytrain1 = []
for i in range(0,len(y_train),15):
    #print(i)
    y1 = y_train[i:i+15]
    ytrain1.append(y1[-1])
    
ytest1 = []
for i in range(0,len(y_test),15):
    #print(i)
    y1 = y_test[i:i+15]
    ytest1.append(y1[-1])
    
ytrain1 = np.array(ytrain1)
ytest1 = np.array(ytest1)

Xtrain = np.reshape(X_train, (-1, 15, X_train.shape[1]))
Xtest = np.reshape(X_test, (-1, 15, X_test.shape[1]))

In [14]:
from sklearn.metrics import roc_auc_score

In [16]:
#Building the LSTM model
X = Input(shape=(None, 1), name='X')
mX = Masking()(X)
lstm = Bidirectional(LSTM(units=512,activation='tanh',return_sequences=True,recurrent_dropout=0.5,dropout=0.3))
mX = lstm(mX)
L = LSTM(units=64,activation='tanh',return_sequences=False)(mX)
y = Dense(1, activation="sigmoid")(L)
outputs = [y]
inputs = [X]
model = Model(inputs,outputs)
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])


#fitting the model
model.fit(Xtrain, ytrain1, batch_size=60 ,validation_split=0.15,epochs=38,callbacks=[es])
#predictions
y_pred = model.predict(Xtest)
#y_pred = y_pred.round()
y_test = np.array(ytest1)
y_pred = np.array(y_pred)
y_test = pd.DataFrame(y_test)
y_test = np.array(y_test)

def acc(x):
    if x>0.5:
        return 1
    else:
        return 0

y_model=[]
for i in y_pred:
    y_model.append(acc(i))
y_answer=[]
for j in y_test:
    y_answer.append(acc(j))

auc_roc = roc_auc_score(y_answer,y_pred)

Train on 29202 samples, validate on 5154 samples
Epoch 1/38
29202/29202 [==============================] - 54s 2ms/step - loss: 0.6829 - acc: 0.5592 - val_loss: 0.8110 - val_acc: 0.0386
Epoch 2/38
29202/29202 [==============================] - 50s 2ms/step - loss: 0.6807 - acc: 0.5645 - val_loss: 0.7870 - val_acc: 0.0386
Epoch 3/38
29202/29202 [==============================] - 50s 2ms/step - loss: 0.6797 - acc: 0.5660 - val_loss: 0.8114 - val_acc: 0.0388
Epoch 4/38
29202/29202 [==============================] - 48s 2ms/step - loss: 0.6795 - acc: 0.5650 - val_loss: 0.8020 - val_acc: 0.0388
Epoch 5/38
29202/29202 [==============================] - 48s 2ms/step - loss: 0.6794 - acc: 0.5664 - val_loss: 0.7923 - val_acc: 0.0388
Restoring model weights from the end of the best epoch
Epoch 00005: early stopping


In [18]:
death = gs[gs['dischargestatus']==1]

In [20]:
dis = gs[gs['dischargestatus']==0]

In [27]:
#% Imputation Death
print((len(death[death['pulserate']==-999])/len(death))*100)

2.4132791438275274


In [28]:
#% Imputation Discharge
print((len(dis[dis['pulserate']==-999])/len(dis))*100)

0.0010859620401968849


In [29]:
cols_to_use = ['uhid','heartrate', 'dischargestatus']

In [30]:
gd = gs[cols_to_use]

In [31]:
def range_finder(x):
    length = x
    fractional = (x/15.0) - math.floor(x/15.0)
    return int(round(fractional*15))

In [32]:
final_df = pd.DataFrame(columns=gd.columns)
for i in gd.uhid.unique():
    x = gd[gd['uhid']==i]
    x = x[range_finder(len(x)):len(x)]
    
    final_df = final_df.append(x,ignore_index=True)
    

In [33]:
final_df.fillna(-999,inplace=True)
train = final_df[:515340]
test = final_df[515340:]

In [34]:
y_train = train['dischargestatus']
X_train = train.drop('dischargestatus',axis=1)
X_train = X_train.drop('uhid',axis=1)
#X_train = X_train.drop('visittime',axis=1)

y_test = test['dischargestatus']
X_test = test.drop('dischargestatus',axis=1)
X_test = X_test.drop('uhid',axis=1)
#X_test = X_test.drop('startdate',axis=1)

In [35]:
#converting the data into a numpy array
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

ytrain1 = []
for i in range(0,len(y_train),15):
    #print(i)
    y1 = y_train[i:i+15]
    ytrain1.append(y1[-1])
    
ytest1 = []
for i in range(0,len(y_test),15):
    #print(i)
    y1 = y_test[i:i+15]
    ytest1.append(y1[-1])
    
ytrain1 = np.array(ytrain1)
ytest1 = np.array(ytest1)

Xtrain = np.reshape(X_train, (-1, 15, X_train.shape[1]))
Xtest = np.reshape(X_test, (-1, 15, X_test.shape[1]))

In [37]:
#Building the LSTM model
X = Input(shape=(None, 1), name='X')
mX = Masking()(X)
lstm = Bidirectional(LSTM(units=512,activation='tanh',return_sequences=True,recurrent_dropout=0.5,dropout=0.3))
mX = lstm(mX)
L = LSTM(units=64,activation='tanh',return_sequences=False)(mX)
y = Dense(1, activation="sigmoid")(L)
outputs = [y]
inputs = [X]
model = Model(inputs,outputs)
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])


#fitting the model
model.fit(Xtrain, ytrain1, batch_size=60 ,validation_split=0.15,epochs=38,callbacks=[es])
#predictions
y_pred = model.predict(Xtest)
#y_pred = y_pred.round()
y_test = np.array(ytest1)
y_pred = np.array(y_pred)
y_test = pd.DataFrame(y_test)
y_test = np.array(y_test)

def acc(x):
    if x>0.5:
        return 1
    else:
        return 0

y_model=[]
for i in y_pred:
    y_model.append(acc(i))
y_answer=[]
for j in y_test:
    y_answer.append(acc(j))

auc_roc = roc_auc_score(y_answer,y_pred)

Train on 29202 samples, validate on 5154 samples
Epoch 1/38
29202/29202 [==============================] - 52s 2ms/step - loss: 0.6530 - acc: 0.5962 - val_loss: 0.8512 - val_acc: 0.0388
Epoch 2/38
29202/29202 [==============================] - 49s 2ms/step - loss: 0.6501 - acc: 0.6017 - val_loss: 0.8524 - val_acc: 0.0388
Epoch 3/38
29202/29202 [==============================] - 49s 2ms/step - loss: 0.6495 - acc: 0.6016 - val_loss: 0.8698 - val_acc: 0.0409
Epoch 4/38
29202/29202 [==============================] - 49s 2ms/step - loss: 0.6493 - acc: 0.6018 - val_loss: 0.8683 - val_acc: 0.0388
Restoring model weights from the end of the best epoch
Epoch 00004: early stopping


In [38]:
death = gs[gs['dischargestatus']==1]
dis = gs[gs['dischargestatus']==0]

In [41]:
#% Imputation Death
print((len(death[death['heartrate']==-999])/len(death))*100)

12.157345040004126


In [42]:
#% Imputation Discharge
print((len(dis[dis['heartrate']==-999])/len(dis))*100)

0.7050608545978275


In [43]:
auc_roc

0.6962056984958658

In [44]:
cols_to_use = ['uhid','spo2', 'dischargestatus']

In [45]:
gd = gs[cols_to_use]

In [46]:
def range_finder(x):
    length = x
    fractional = (x/15.0) - math.floor(x/15.0)
    return int(round(fractional*15))

In [47]:
final_df = pd.DataFrame(columns=gd.columns)
for i in gd.uhid.unique():
    x = gd[gd['uhid']==i]
    x = x[range_finder(len(x)):len(x)]
    
    final_df = final_df.append(x,ignore_index=True)
    

In [48]:
final_df.fillna(-999,inplace=True)
train = final_df[:515340]
test = final_df[515340:]

In [49]:
y_train = train['dischargestatus']
X_train = train.drop('dischargestatus',axis=1)
X_train = X_train.drop('uhid',axis=1)
#X_train = X_train.drop('visittime',axis=1)

y_test = test['dischargestatus']
X_test = test.drop('dischargestatus',axis=1)
X_test = X_test.drop('uhid',axis=1)
#X_test = X_test.drop('startdate',axis=1)

In [50]:
#converting the data into a numpy array
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

ytrain1 = []
for i in range(0,len(y_train),15):
    #print(i)
    y1 = y_train[i:i+15]
    ytrain1.append(y1[-1])
    
ytest1 = []
for i in range(0,len(y_test),15):
    #print(i)
    y1 = y_test[i:i+15]
    ytest1.append(y1[-1])
    
ytrain1 = np.array(ytrain1)
ytest1 = np.array(ytest1)

Xtrain = np.reshape(X_train, (-1, 15, X_train.shape[1]))
Xtest = np.reshape(X_test, (-1, 15, X_test.shape[1]))

In [51]:
#Building the LSTM model
X = Input(shape=(None, 1), name='X')
mX = Masking()(X)
lstm = Bidirectional(LSTM(units=512,activation='tanh',return_sequences=True,recurrent_dropout=0.5,dropout=0.3))
mX = lstm(mX)
L = LSTM(units=64,activation='tanh',return_sequences=False)(mX)
y = Dense(1, activation="sigmoid")(L)
outputs = [y]
inputs = [X]
model = Model(inputs,outputs)
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])


#fitting the model
model.fit(Xtrain, ytrain1, batch_size=60 ,validation_split=0.15,epochs=38,callbacks=[es])
#predictions
y_pred = model.predict(Xtest)
#y_pred = y_pred.round()
y_test = np.array(ytest1)
y_pred = np.array(y_pred)
y_test = pd.DataFrame(y_test)
y_test = np.array(y_test)

def acc(x):
    if x>0.5:
        return 1
    else:
        return 0

y_model=[]
for i in y_pred:
    y_model.append(acc(i))
y_answer=[]
for j in y_test:
    y_answer.append(acc(j))

auc_roc = roc_auc_score(y_answer,y_pred)

Train on 29202 samples, validate on 5154 samples
Epoch 1/38
29202/29202 [==============================] - 52s 2ms/step - loss: 0.6821 - acc: 0.5584 - val_loss: 0.6784 - val_acc: 0.5927
Epoch 2/38
29202/29202 [==============================] - 50s 2ms/step - loss: 0.6793 - acc: 0.5648 - val_loss: 0.6594 - val_acc: 0.9373
Epoch 3/38
29202/29202 [==============================] - 51s 2ms/step - loss: 0.6782 - acc: 0.5664 - val_loss: 0.6651 - val_acc: 0.8104
Epoch 4/38
29202/29202 [==============================] - 48s 2ms/step - loss: 0.6773 - acc: 0.5682 - val_loss: 0.6322 - val_acc: 0.9777
Epoch 5/38
29202/29202 [==============================] - 48s 2ms/step - loss: 0.6770 - acc: 0.5694 - val_loss: 0.6623 - val_acc: 0.8890
Epoch 6/38
29202/29202 [==============================] - 51s 2ms/step - loss: 0.6752 - acc: 0.5747 - val_loss: 0.4843 - val_acc: 0.9777
Epoch 7/38
29202/29202 [==============================] - 50s 2ms/step - loss: 0.6737 - acc: 0.5790 - val_loss: 0.4468 - val_acc:

In [52]:
#% Imputation Death
print((len(death[death['spo2']==-999])/len(death))*100)

2.2742760026823263


In [78]:
#% Imputation Discharge
print((len(dis[dis['spo2']==-999])/len(dis))*100)

0.0021719240803937698


In [54]:
auc_roc

0.39503299209961856

In [67]:
cols_to_use = ['uhid','ecg_resprate', 'dischargestatus']

In [68]:
gd = gs[cols_to_use]

In [69]:
def range_finder(x):
    length = x
    fractional = (x/15.0) - math.floor(x/15.0)
    return int(round(fractional*15))

In [70]:
final_df = pd.DataFrame(columns=gd.columns)
for i in gd.uhid.unique():
    x = gd[gd['uhid']==i]
    x = x[range_finder(len(x)):len(x)]
    
    final_df = final_df.append(x,ignore_index=True)
    

In [71]:
final_df.fillna(-999,inplace=True)
train = final_df[:515340]
test = final_df[515340:]

In [72]:
y_train = train['dischargestatus']
X_train = train.drop('dischargestatus',axis=1)
X_train = X_train.drop('uhid',axis=1)
#X_train = X_train.drop('visittime',axis=1)

y_test = test['dischargestatus']
X_test = test.drop('dischargestatus',axis=1)
X_test = X_test.drop('uhid',axis=1)
#X_test = X_test.drop('startdate',axis=1)

In [73]:
#converting the data into a numpy array
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

ytrain1 = []
for i in range(0,len(y_train),15):
    #print(i)
    y1 = y_train[i:i+15]
    ytrain1.append(y1[-1])
    
ytest1 = []
for i in range(0,len(y_test),15):
    #print(i)
    y1 = y_test[i:i+15]
    ytest1.append(y1[-1])
    
ytrain1 = np.array(ytrain1)
ytest1 = np.array(ytest1)

Xtrain = np.reshape(X_train, (-1, 15, X_train.shape[1]))
Xtest = np.reshape(X_test, (-1, 15, X_test.shape[1]))

In [74]:
#Building the LSTM model
X = Input(shape=(None, 1), name='X')
mX = Masking()(X)
lstm = Bidirectional(LSTM(units=512,activation='tanh',return_sequences=True,recurrent_dropout=0.5,dropout=0.3))
mX = lstm(mX)
L = LSTM(units=64,activation='tanh',return_sequences=False)(mX)
y = Dense(1, activation="sigmoid")(L)
outputs = [y]
inputs = [X]
model = Model(inputs,outputs)
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])


#fitting the model
model.fit(Xtrain, ytrain1, batch_size=60 ,validation_split=0.15,epochs=38,callbacks=[es])
#predictions
y_pred = model.predict(Xtest)
#y_pred = y_pred.round()
y_test = np.array(ytest1)
y_pred = np.array(y_pred)
y_test = pd.DataFrame(y_test)
y_test = np.array(y_test)

def acc(x):
    if x>0.5:
        return 1
    else:
        return 0

y_model=[]
for i in y_pred:
    y_model.append(acc(i))
y_answer=[]
for j in y_test:
    y_answer.append(acc(j))

auc_roc = roc_auc_score(y_answer,y_pred)

Train on 29202 samples, validate on 5154 samples
Epoch 1/38
29202/29202 [==============================] - 53s 2ms/step - loss: 0.2737 - acc: 0.9081 - val_loss: 0.2428 - val_acc: 0.8855
Epoch 2/38
29202/29202 [==============================] - 49s 2ms/step - loss: 0.2664 - acc: 0.9110 - val_loss: 0.2382 - val_acc: 0.8857
Epoch 3/38
29202/29202 [==============================] - 48s 2ms/step - loss: 0.2658 - acc: 0.9114 - val_loss: 0.2515 - val_acc: 0.8857
Epoch 4/38
29202/29202 [==============================] - 48s 2ms/step - loss: 0.2649 - acc: 0.9117 - val_loss: 0.2342 - val_acc: 0.8857
Epoch 5/38
29202/29202 [==============================] - 52s 2ms/step - loss: 0.2652 - acc: 0.9114 - val_loss: 0.2410 - val_acc: 0.8857
Epoch 6/38
29202/29202 [==============================] - 49s 2ms/step - loss: 0.2645 - acc: 0.9116 - val_loss: 0.2401 - val_acc: 0.8857
Epoch 7/38
29202/29202 [==============================] - 49s 2ms/step - loss: 0.2644 - acc: 0.9117 - val_loss: 0.2514 - val_acc:

In [75]:
#% Imputation Death
print((len(death[death['ecg_resprate']==-999])/len(death))*100)

21.926116572595205


In [76]:
#% Imputation Discharge
print((len(dis[dis['ecg_resprate']==-999])/len(dis))*100)

98.0297933685728


In [77]:
auc_roc

0.8256769249629745